In [1]:
from z3 import *
from data import hint
from tmp import construct, taps
from copy import deepcopy
from Crypto.Util.number import long_to_bytes
from tqdm import trange

In [2]:

def xors(lst):
    ret = lst[0]
    for x in lst[1:]:
        ret = Xor(ret, x)
    return ret



def genmat(start=256, end=256 + 70):
    mat = []
    for i in trange(start, end):
        cur = 71 * (i + 1) - 1
        row = construct(taps, 64, cur)[1][0]
        # print(len(row), row)
        mat.append(row)
    # for row in mat:
    #     print(row)
    return mat




In [31]:


eq = 20
mat = genmat(256, 256 + eq)
xs = [Bool(f"x{i}") for i in range(64)]
s = Solver()

100%|██████████| 20/20 [00:06<00:00,  3.29it/s]


In [32]:

for i in range(eq):
    par = [xs[j] for j in range(64) if mat[i][j]]
    print(i, par)
    clause = xors(par)
    if hint[256 + i]:
        s.add(clause)
    else:
        s.add(Not(clause))


0 [x0, x3, x5, x6, x8, x10, x11, x12, x14, x16, x17, x19, x20, x21, x25, x26, x27, x28, x29, x32, x35, x36, x38, x39, x41, x43, x47, x48, x49, x50, x54, x55, x56, x59, x61, x62]
1 [x1, x5, x7, x8, x9, x10, x13, x15, x17, x19, x21, x22, x26, x27, x31, x32, x40, x43, x44, x46, x47, x53, x54, x55, x56, x57, x59, x60, x61, x62, x63]
2 [x2, x3, x6, x7, x8, x11, x12, x14, x15, x19, x21, x25, x27, x31, x33, x34, x35, x36, x37, x38, x40, x41, x42, x44, x45, x47, x50, x51, x52, x53, x54, x55, x59, x60, x63]
3 [x0, x1, x5, x6, x7, x10, x14, x16, x21, x23, x24, x26, x27, x28, x29, x30, x31, x34, x36, x42, x43, x45, x48, x49, x51, x52, x55, x57, x58, x60, x61, x63]
4 [x1, x3, x5, x8, x11, x12, x14, x15, x16, x18, x20, x22, x24, x25, x26, x27, x29, x30, x31, x32, x34, x35, x37, x42, x44, x45, x53, x54, x55, x60, x63]
5 [x1, x2, x4, x5, x6, x7, x8, x9, x10, x11, x12, x16, x19, x22, x23, x25, x29, x31, x32, x33, x34, x35, x36, x40, x41, x42, x43, x45, x46, x47, x48, x52, x53, x55, x56, x57, x59, x60,

In [33]:

if s.check().r == 1:
    solution = {}
    for x in s.model():
        solution[int(str(x)[1:])] = s.model()[x]
    sol = [1 if solution[i] else 0 for i in range(64)]
else:
    print("unsat")

In [34]:


def resolve(sol):
    mat = genmat(0, 256)
    stream = []
    for i in range(256):
        cur = sum([sol[j] * mat[i][j] for j in range(64)]) & 1
        stream.append(cur)

    print(len(stream), stream)
    print(len(hint), hint)
    flag = []
    for i in range(256):
        flag.append(stream[i] ^ hint[i])
    flag = long_to_bytes(int("".join(map(str, flag)), 2))
    print(flag)


# resolve(sol)

100%|██████████| 256/256 [01:16<00:00,  3.35it/s]

256 [1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0]
326 [0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 

In [3]:
import numpy as np

In [23]:

mat = genmat(256, 256 + 64)
# mat_inv = np.linalg.inv(mat)
# print(mat.shape)

  0%|          | 0/64 [00:00<?, ?it/s]

100%|██████████| 64/64 [00:20<00:00,  3.14it/s]


In [8]:
print(mat)

[[1 0 0 ... 1 1 0]
 [0 1 0 ... 1 1 1]
 [0 0 1 ... 0 0 1]
 ...
 [1 0 0 ... 1 0 1]
 [1 0 1 ... 0 1 0]
 [1 0 0 ... 1 1 0]]


In [28]:
def merge(r1,r2):
    ret = []
    for i in range(65):
        ret.append(r1[i] ^ r2[i])
    return ret

sol = deepcopy(mat)
for i in range(64):
    sol[i].append(hint[256+i])
print(np.matrix(sol).shape)

(64, 65)


In [34]:
for i in range(64):
    if sol[i][i] == 0:
        for j in range(i+1, 64):
            if sol[j][i] == 1:
                sol[i], sol[j] = sol[j], sol[i]
                break
    assert sol[i][i] == 1
    for j in range(i+1, 64):
        if sol[j][i] == 1:
            sol[j] = merge(sol[i], sol[j])

for i in range(63, -1, -1):
    for j in range(i-1, -1, -1):
        if sol[j][i] == 1:
            sol[j] = merge(sol[i], sol[j])
key = [sol[i][64] for i in range(64)]
print(sol)
print(np.matrix(sol))
print(sol)
print(key)

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0

In [35]:

mat = genmat(0, 256)
stream = []
for i in range(256):
    cur = sum([key[j] * mat[i][j] for j in range(64)]) & 1
    stream.append(cur)

print(len(stream), stream)
print(len(hint), hint)

flag = []
for i in range(256):
    flag.append(stream[i] ^ hint[i])
flag = long_to_bytes(int("".join(map(str, flag)), 2))
print(flag)


100%|██████████| 256/256 [01:11<00:00,  3.56it/s]

256 [0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0]
326 [0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 